In [3]:
import pandas as pd

In [4]:
results = pd.read_csv('results.csv', dtype={'season':str})

In [5]:
results.head()

,season,round,tie,winner,pk,agr,aet,result
0,2008,first,arsenal-milan,arsenal,False,False,False,arsenal (2-0) milan
1,2008,first,celtic-barcelona,barcelona,False,False,False,celtic (2-4) barcelona
2,2008,first,fenerbahce-sevilla,fenerbahce,True,False,True,"fenerbahce (5-5 aet) sevilla, fenerbahce won o..."
3,2008,first,liverpool-inter,liverpool,False,False,False,liverpool (3-0) inter
4,2008,first,lyon-united,united,False,False,False,lyon (1-2) united


In [6]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 8 columns):
season    162 non-null object
round     162 non-null object
tie       162 non-null object
winner    162 non-null object
pk        162 non-null bool
agr       162 non-null bool
aet       162 non-null bool
result    162 non-null object
dtypes: bool(3), object(5)
memory usage: 6.9+ KB


In [7]:
goles = pd.read_csv('goles - liga de campeones - Sheet1.tsv', sep='\t', dtype=str)

In [8]:
goles['season'].fillna(method='pad', inplace=True)
goles['round'].fillna(method='pad', inplace=True)
goles['tie'].fillna(method='pad', inplace=True)
goles['game'].fillna(method='pad', inplace=True)

In [9]:
goles.head()

,season,round,tie,game,minute,away,extra,shootout,note
0,2017,first,city-monaco,1,26,NaN,NaN,NaN,NaN
1,2017,first,city-monaco,1,32,a,NaN,NaN,NaN
2,2017,first,city-monaco,1,40,a,NaN,NaN,NaN
3,2017,first,city-monaco,1,58,NaN,NaN,NaN,NaN
4,2017,first,city-monaco,1,61,a,NaN,NaN,NaN


In [10]:
goles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961 entries, 0 to 960
Data columns (total 9 columns):
season      961 non-null object
round       961 non-null object
tie         961 non-null object
game        961 non-null object
minute      936 non-null object
away        368 non-null object
extra       12 non-null object
shootout    7 non-null object
note        0 non-null object
dtypes: object(9)
memory usage: 67.6+ KB


In [13]:
def make_minutes(row):
    '''
    1-45: Game 1 first half (1-45)
    46: Game 1 first-half stoppage time
    47-91: Game 1 second-half (46-90)
    92: Game 1 second-half stoppage time
    
    93-137: Game 2 first half (1-45)
    138: Game 2 first-half stoppage time
    139-183: Game 2 second half (46-90)
    184: Game 2 second-half stoppage time
    
    185-199: Game 2 extra time first half (91-105)
    200: Game 2 extra time first half stoppage time
    201-215: Game 2 extra time second half (106-120)
    216: Game 2 extra time second half stoppage time
    
    217: Penalty shootout
    '''
    
    g = row.game
    m = row.minute
        
    if g == '1':
        if pd.isnull(m):
            return m
        if '45+' in m:
            return 46
        if '90+' in m:
            return 92
        if int(m) >= 1 and int(m) <= 45:
            return int(m)
        if int(m) >= 46 and int(m) <= 90:
            return int(m) + 1
        
    if g == '2':
        if pd.isnull(m):
            return m
        if '45+' in m:
            return 138
        if '90+' in m:
            return 184
        if '105+' in m:
            return 200
        if '120+' in m:
            return 216
        if 'pk' in m:
            return 217
        if int(m) >= 1 and int(m) <= 45:
            return int(m) + 92
        if int(m) >= 46 and int(m) <= 90:
            return int(m) + 93
        if int(m) >= 91 and int(m) <= 105:
            return int(m) + 94
        if int(m) >= 106 and int(m) <= 120:
            return int(m) + 95

In [14]:
goles['made_minute'] = goles.apply(make_minutes, axis=1)

In [15]:
ties = list(goles.groupby(['season','round','tie',]))

In [43]:
ties[50][1]

,season,round,tie,game,minute,away,extra,shootout,note,made_minute
565,2011,qtr,barcelona-shakhtar,1,2,NaN,NaN,NaN,NaN,2.0
566,2011,qtr,barcelona-shakhtar,1,34,NaN,NaN,NaN,NaN,34.0
567,2011,qtr,barcelona-shakhtar,1,53,NaN,NaN,NaN,NaN,54.0
568,2011,qtr,barcelona-shakhtar,1,61,NaN,NaN,NaN,NaN,62.0
569,2011,qtr,barcelona-shakhtar,1,86,NaN,NaN,NaN,NaN,87.0
570,2011,qtr,barcelona-shakhtar,1,60,a,NaN,NaN,NaN,61.0
571,2011,qtr,barcelona-shakhtar,2,43,a,NaN,NaN,NaN,135.0


In [17]:
awaymindiff = {
    i[0]:i[1] for i in
    zip(
        list(range(1,218)),
        list(range(0,92))+list(range(91,-1,-1))+list(range(31,-1,-1))+[0]
    )
}

In [48]:
def create_matrix(df):
    result = df[['season','round','tie']].drop_duplicates().merge(results)
    et = (result['aet'].sum() > 0)
    made_min_max = 184 if not et else 217
    
    goals = df[['made_minute','away']]
    goals = goals[goals.made_minute.notnull()]
    goals.away.fillna('h', inplace=True)
    
    minutes = goals.merge(
        pd.DataFrame({'made_minute':range(1,made_min_max+1)}),
        how='right'
    ).sort_values('made_minute')
    
    minutes['game'] = minutes.made_minute.apply(lambda x: 1 if x <= 92 else 2)
    
    minutes['t1goaldiff'] = (
        ((minutes['away'] == 'h') & (minutes['made_minute'] <= 92)) |
        ((minutes['away'] == 'a') & (minutes['made_minute'] > 92))
    ).cumsum() - (
        ((minutes['away'] == 'a') & (minutes['made_minute'] <= 92)) |
        ((minutes['away'] == 'h') & (minutes['made_minute'] > 92))
    ).cumsum()

    minutes['t1awaygoaldiff'] = (
        (minutes['away'] == 'a') & (minutes['made_minute'] > 92)
    ).cumsum() - (
        (minutes['away'] == 'a') & (minutes['made_minute'] <= 92)
    ).cumsum()
    
    minutes['t1home'] = (minutes.made_minute <= 92).astype(int)
    
    minutes['stoppagetime'] = minutes.made_minute.apply(lambda x: int(x in [46,92,138,184,200,216]))
    minutes['extratime'] = (minutes.made_minute >= 185).astype(int)
    
#     minutes['t1awaymindiff'] = minutes.made_minute.apply(lambda x: awaymindiff[x])
    
    return minutes

In [49]:
allminutes = goles\
.groupby(['season','round','tie',])\
.apply(create_matrix)\
.reset_index()\
.drop('level_3', axis=1)\

allminutes['t1'] = allminutes.tie.apply(lambda x: x.split('-')[0])
allminutes['t2'] = allminutes.tie.apply(lambda x: x.split('-')[1])

allminutes = allminutes.merge(results[['season','round','tie','winner']])

allminutes['t1win'] = (allminutes.t1 == allminutes.winner).astype(int)

In [50]:
allminutes.head()

,season,round,tie,made_minute,away,game,t1goaldiff,t1awaygoaldiff,t1home,stoppagetime,extratime,t1,t2,winner,t1win
0,2008,first,arsenal-milan,1.0,NaN,1,0,0,1,0,0,arsenal,milan,arsenal,1
1,2008,first,arsenal-milan,2.0,NaN,1,0,0,1,0,0,arsenal,milan,arsenal,1
2,2008,first,arsenal-milan,3.0,NaN,1,0,0,1,0,0,arsenal,milan,arsenal,1
3,2008,first,arsenal-milan,4.0,NaN,1,0,0,1,0,0,arsenal,milan,arsenal,1
4,2008,first,arsenal-milan,5.0,NaN,1,0,0,1,0,0,arsenal,milan,arsenal,1


In [51]:
allminutes.tail()

,season,round,tie,made_minute,away,game,t1goaldiff,t1awaygoaldiff,t1home,stoppagetime,extratime,t1,t2,winner,t1win
30236,2019,first,united-psg,180.0,NaN,2,-1,0,0,0,0,united,psg,united,1
30237,2019,first,united-psg,181.0,NaN,2,-1,0,0,0,0,united,psg,united,1
30238,2019,first,united-psg,182.0,NaN,2,-1,0,0,0,0,united,psg,united,1
30239,2019,first,united-psg,183.0,NaN,2,-1,0,0,0,0,united,psg,united,1
30240,2019,first,united-psg,184.0,a,2,0,1,0,1,0,united,psg,united,1


In [42]:
len(allminutes)

30241

In [63]:
odds = pd.read_csv('game-odds-xwalk.csv', dtype={'season':str})\
.drop(['hometeam','awayteam'], axis=1)

In [64]:
odds.head()

,game,season,tie,homewin,draw,awaywin
0,1,2008,arsenal-milan,0.452474,0.314634,0.232891
1,2,2008,arsenal-milan,0.428566,0.317081,0.254352
2,1,2008,celtic-barcelona,0.234004,0.302405,0.463592
3,2,2008,celtic-barcelona,0.821256,0.146369,0.032374
4,1,2008,fenerbahce-sevilla,0.342100,0.309465,0.348435


In [71]:
odds['homewin'] = odds['homewin'].fillna(0.33)
odds['draw'] = odds['draw'].fillna(0.33)
odds['awaywin'] = odds['awaywin'].fillna(0.33)

In [103]:
allminutesodds = allminutes\
.merge(odds[odds.game==1], how='left', on=['tie','season','game'])\
.merge(odds[odds.game==2], how='left', on=['tie','season','game'], suffixes=['g1','g2'])

In [104]:
allminutesodds['homewing1'] = allminutesodds['homewing1'].fillna(method='ffill')
allminutesodds['drawg1'] = allminutesodds['drawg1'].fillna(method='ffill')
allminutesodds['awaywing1'] = allminutesodds['awaywing1'].fillna(method='ffill')
allminutesodds['homewing2'] = allminutesodds['homewing2'].fillna(0)
allminutesodds['drawg2'] = allminutesodds['drawg2'].fillna(0)
allminutesodds['awaywing2'] = allminutesodds['awaywing2'].fillna(0)


In [105]:
allminutesodds.head()

,season,round,tie,made_minute,away,game,t1goaldiff,t1awaygoaldiff,t1home,stoppagetime,...,t1,t2,winner,t1win,homewing1,drawg1,awaywing1,homewing2,drawg2,awaywing2
0,2008,first,arsenal-milan,1.0,NaN,1,0,0,1,0,...,arsenal,milan,arsenal,1,0.452474,0.314634,0.232891,0.0,0.0,0.0
1,2008,first,arsenal-milan,2.0,NaN,1,0,0,1,0,...,arsenal,milan,arsenal,1,0.452474,0.314634,0.232891,0.0,0.0,0.0
2,2008,first,arsenal-milan,3.0,NaN,1,0,0,1,0,...,arsenal,milan,arsenal,1,0.452474,0.314634,0.232891,0.0,0.0,0.0
3,2008,first,arsenal-milan,4.0,NaN,1,0,0,1,0,...,arsenal,milan,arsenal,1,0.452474,0.314634,0.232891,0.0,0.0,0.0
4,2008,first,arsenal-milan,5.0,NaN,1,0,0,1,0,...,arsenal,milan,arsenal,1,0.452474,0.314634,0.232891,0.0,0.0,0.0


In [106]:
allminutesodds.tail()

,season,round,tie,made_minute,away,game,t1goaldiff,t1awaygoaldiff,t1home,stoppagetime,...,t1,t2,winner,t1win,homewing1,drawg1,awaywing1,homewing2,drawg2,awaywing2
30236,2019,first,united-psg,180.0,NaN,2,-1,0,0,0,...,united,psg,united,1,0.412593,0.278619,0.308788,0.648003,0.207221,0.144775
30237,2019,first,united-psg,181.0,NaN,2,-1,0,0,0,...,united,psg,united,1,0.412593,0.278619,0.308788,0.648003,0.207221,0.144775
30238,2019,first,united-psg,182.0,NaN,2,-1,0,0,0,...,united,psg,united,1,0.412593,0.278619,0.308788,0.648003,0.207221,0.144775
30239,2019,first,united-psg,183.0,NaN,2,-1,0,0,0,...,united,psg,united,1,0.412593,0.278619,0.308788,0.648003,0.207221,0.144775
30240,2019,first,united-psg,184.0,a,2,0,1,0,1,...,united,psg,united,1,0.412593,0.278619,0.308788,0.648003,0.207221,0.144775


In [107]:
allminutesodds.to_csv('minute-matrix.csv', index=False)